In [2]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from PIL.Image import open as image_open

# Cata-Dado

Notebook para testes de uma aplicação que permite obter os dados de um gráfico a partir da sua imagem.

Uma vez aberta a imagem, devemos ser capazes de clicar em quatro pontos de referência nos eixos e, dando os valores de referência, recriar os eixos a partir de uma relação linear.

Depois desse processo de calibração, devemos ser capazes de, clicando nos pontos da imagem, criar uma tabela com os dados retirados da imagem.

Configurações para a imagem do matplotlib:

In [3]:
plt.rcParams['backend'] = 'TkAgg' # backen do matplotlib
plt.rcParams["figure.figsize"] = [5, 5] # tamanho da figura
plt.rcParams["figure.autolayout"] = True 

A função abaixo coleta os pontos da imagem que são clicados (os valores dos pixels correspondentes às posições horizontal e vertical), pede um valor que corresponde ao valor real dos dados da imagem e salva os dados em uma lista chamada pontos.

Coletando 4 pontos referentes ao eixos (na ordem: 2 horizontais depois 2 verticais) podemos fazer a correspondência entre o valor da imagem e o valor real do dados usando uma função linear.

Criamos um gráfico quadrático para testar a função. Posteriormente, onde criamos um gráfico, vamos abrir uma imagem.

In [4]:
pontos = [] # cria uma lista vazia para armazenar os dados de calibração

# Função que imprime as coordenadas dos cliques
def onclick(event):
    global pontos
    valor = input('descreva') # usamos para escrever os valores de referência
    pontos.append((event.xdata, event.ydata, valor))

# Criando uma figura para testar a função
fig, ax = plt.subplots()

ax.scatter([1,2,3,4,5], # relação quadrática
        [1,4,9,16,25])

# Vincula o button_press_event com a função onclick
fig.canvas.mpl_connect('button_press_event', onclick)

# Mostra a figura
plt.show()

descreva 1
descreva 5
descreva 5
descreva 25


Olhando como ficou a lista de pontos com as referências para calibração:

In [5]:
pontos

[(1.0041052036909173, -0.006099838167558325, '1'),
 (5.011085305379449, -0.06525581974355754, '5'),
 (0.8151905090703659, 5.022158595792357, '5'),
 (0.8151905090703659, 25.07603635005602, '25')]

Tranformando a lista de pontos em uma array de floats e renomeando para Pontos.

In [17]:
Pontos = np.array(pontos).astype('float')
Pontos

array([[ 1.00410520e+00, -6.09983817e-03,  1.00000000e+00],
       [ 5.01108531e+00, -6.52558197e-02,  5.00000000e+00],
       [ 8.15190509e-01,  5.02215860e+00,  5.00000000e+00],
       [ 8.15190509e-01,  2.50760364e+01,  2.50000000e+01]])

Separando a lista de pontos em coordenadas verticais (y) e horizontais (x). As respectivas referências para calibração são chamadas de Y e X.

Como X e Y são strings, transformamos para float para seguir os cálculos.

In [18]:
x_0 = Pontos[0][0]
x_1 = Pontos[1][0]
y_0 = Pontos[2][1]
y_1 = Pontos[3][1]
X_0 = np.float64(pontos[0][2])
X_1 = np.float64(pontos[1][2])
Y_0 = np.float64(pontos[2][2])
Y_1 = np.float64(pontos[3][2])

Abaixo, estamos construindo uma relação linear entre os pontos  clicados em unidades de pixel e os valores de referência nas unidades do gráfico.

Os valores de a e b se referem respectivamente aos coeficientes angular e linear da relação linear entre os pontos em unidade de pixel (x, y) e a na unidade real do gráfico (X, Y). Os íncices h e v se referem respectivamente aos eixos horizontal e vertical da imagem.

In [19]:
a_h = (X_1-X_0)/(x_1-x_0)

b_h = ((X_0 - a_h*x_0)+(X_1 - a_h*x_1) )/2

a_v = (Y_1-Y_0)/(y_1-y_0)

b_v = ((Y_0 - a_v*y_0)+(Y_1 - a_v*y_1) )/2

Nesse ponto, já podemos calibrar os dados. Então, retornamos à figura para coletar os pontos da imagem. Usamos a mesma função utilizada algumas células acima e a usamos o mesmo gráfico criado para os testes.

A referência será colocada apenas para a conferência do método, não será utilizada nos cálculos porque os dados para a calibração já foram coletados.

In [30]:
pontos = [] # cria uma lista vazia para armazenar os dados de calibração

# Função que imprime as coordenadas dos cliques
def onclick(event):
    global pontos
    valor = input('descreva') # usamos para escrever os valores de referência
    pontos.append((event.xdata, event.ydata,valor))

# Criando uma figura para testar a função
fig, ax = plt.subplots()

ax.scatter([1,2,3,4,5],
        [1,4,9,16,25])

# Vincula o button_press_event com a função onclick
fig.canvas.mpl_connect('button_press_event', onclick)

# Mostra a figura
plt.show()

descreva 1
descreva 4
descreva 9
descreva 16
descreva 25


Vamos olhar os pontos e tranformar em uma array de floats:

In [35]:
Pontos = np.array(pontos).astype('float')
Pontos

array([[ 1.01404808,  0.8220839 ,  1.        ],
       [ 1.99839307,  4.13481887,  4.        ],
       [ 3.00262382,  9.10392132,  9.        ],
       [ 3.99691168, 16.14348313, 16.        ],
       [ 5.01108531, 25.01688037, 25.        ]])

Vamos separar os Pontos em coordenadas x (horizontais) e y (verticais). Guardamos também o gabarito para posterior conferência.

In [55]:
x = Pontos[0:, 0]
y = Pontos[0:, 1]
gabarito_Y = Pontos[0:, 2]
x, y, gabarito_Y

(array([1.01404808, 1.99839307, 3.00262382, 3.99691168, 5.01108531]),
 array([ 0.8220839 ,  4.13481887,  9.10392132, 16.14348313, 25.01688037]),
 array([ 1.,  4.,  9., 16., 25.]))

Vamos usar os coeficientes angular e linear calculados anteriromente para construir a relação de calibração.

In [56]:
X = a_h*x+b_h
Y = a_v*y+b_v

Com isso, podemos ver a lista de X e Y dados pela calibração e conferir com a relação quadrática que construímos.

In [57]:
X

array([1.00992556, 1.99255583, 2.99503722, 3.98759305, 5.        ])

In [66]:
Y

array([ 0.81120944,  4.11504425,  9.07079646, 16.09144543, 24.94100295])

Vamos comparar o valor de Y real com o encontrado depois da calibração.

Calculando o erro percentual ponto a ponto na coordenada Y:

In [67]:
abs((Y-gabarito_Y)/gabarito_Y)

array([0.18879056, 0.02876106, 0.00786627, 0.00571534, 0.00235988])

Calculando a soma dos erros percentuais.

In [68]:
abs((Y-gabarito_Y)/gabarito_Y).sum()

0.23349311701081557

Para a coordenada x, temo um erro percentual:


In [69]:
gabarito_X = np.array([1,2,3,4,5])
abs((X-gabarito_X)/gabarito_X)

array([0.00992556, 0.00372208, 0.00165426, 0.00310174, 0.        ])

Somando os erros percentuais:

In [72]:
abs((X-gabarito_X)/gabarito_X).sum()

0.018403639371381196

Encontramos um resultado satisfatório mesmo sem muito cuidado no clique.